In [ ]:
%%capture
!pip install datasets transformers

In [ ]:

import torch
import time
import datetime

import random
from random import seed

import numpy as np
from sklearn.metrics import accuracy_score
import torch.utils.data as Data

from datasets import load_dataset
from transformers import AutoTokenizer
def format_time(elapsed):
    """Takes a time in seconds and returns a string hh:mm:ss."""
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:

def get_device():
    if torch.backends.cuda.is_built():
        print("CUDA")
        device = torch.device("cuda")
    elif torch.backends.mps.is_built():
        print("mps")
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
        raise Exception("GPU is not avalaible!")
    return device


In [ ]:
from transformers import BertForSequenceClassification
from torch.nn import CrossEntropyLoss, MSELoss
import torch

class MyBertMaxPoolClassifier(BertForSequenceClassification):
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
    ):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=False,
        )

        last_hidden_state = outputs[0]  

        extended_attention_mask = attention_mask.unsqueeze(-1).bool()
        masked_hidden_state = last_hidden_state.masked_fill(~extended_attention_mask, float('-inf'))

        
        pooled_output, _ = torch.max(masked_hidden_state, dim=1)

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]

       
        if labels is not None:
            if self.num_labels == 1:
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs


In [ ]:
from transformers.modeling_outputs import SequenceClassifierOutput
import torch
import torch.nn as nn

class AttentionPoolBertClassifier(BertForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        
        self.attention_layer = nn.Linear(config.hidden_size, 1)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None
    ):
        
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict
        )

        
        last_hidden_state = outputs[0]

        
        
        attn_scores = self.attention_layer(last_hidden_state)
        attn_scores = attn_scores.masked_fill(attention_mask.unsqueeze(-1) == 0, -1e9)
        attn_weights = torch.softmax(attn_scores, dim=1)
        pooled_output = torch.sum(last_hidden_state * attn_weights, dim=1)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if self.num_labels == 1:
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states if output_hidden_states else None,
            attentions=outputs.attentions if output_attentions else None,
        )

In [ ]:
def train_eval_loop(
    model, loader, optimizer, scheduler, device, n_epochs=2, seed_val=42
):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    loss_values = []
    t00 = time.time()

    for epoch_i in range(0, n_epochs):
        print("")
        print("======== Epoch {:} / {:} ========".format(epoch_i + 1, n_epochs))
        print("Training...")

        t0 = time.time()
        total_loss = 0
        model.train()

        for step, batch in enumerate(loader["train"]):
            b_input_ids, b_input_mask, b_token_type_ids, b_labels = tuple(t.to(device) for t in batch)
            model.zero_grad()

            outputs = model(
                input_ids=b_input_ids,
                attention_mask=b_input_mask,
                token_type_ids=b_token_type_ids,
                labels=b_labels
            )
            loss = outputs[0]

            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_loss / len(loader["train"])
        loss_values.append(avg_train_loss)

        print("\nAverage training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(format_time(time.time() - t00)))

        print("\nRunning Validation...")
        t0 = time.time()
        model.eval()
        val_acc, nb_eval_steps = 0, 0

        for batch in loader["validation"]:
            b_input_ids, b_input_mask, b_token_type_ids, b_labels = tuple(t.to(device) for t in batch)

            with torch.no_grad():
                logits = model(
                    input_ids=b_input_ids,
                    attention_mask=b_input_mask,
                    token_type_ids=b_token_type_ids
                )[0]

            logits = logits.detach().cpu().numpy()
            logits = np.argmax(logits, axis=1).flatten()
            label_ids = b_labels.to("cpu").numpy()

            val_acc += accuracy_score(logits, label_ids)
            nb_eval_steps += 1

        val_acc = 100 * (val_acc / nb_eval_steps)
        print("  Validation ACC: {0:.2f}".format(val_acc))
        print("  Validation took: {:}".format(format_time(time.time() - t0)))

    return val_acc, loss_values


In [ ]:
def init_loader(max_length=16, batch_size=32, test_size=0.2, random_state=2023):
    model_checkpoint = "bert-base-uncased"

    dataset = load_dataset("glue", "rte")


    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

    df_s, x, y = {}, {}, {}
    input_ids, attention_mask = {}, {}
    token_type_ids = {}
    datasets, loader = {}, {}

    max_length = 128

    for split in ["train", "validation", "test"]:

        df_s[split] = dataset[split].to_pandas()

        premise = dataset[split]["sentence1"]
        hypothesis = dataset[split]["sentence2"]
        y[split] = dataset[split]["label"]

        x[split] = list(zip(premise, hypothesis))

        input= tokenizer(
            premise,
            hypothesis,
            max_length=max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
            return_token_type_ids=True,
        )

        input_ids[split], attention_mask[split],token_type_ids[split]  = input.input_ids, input.attention_mask,input.token_type_ids

        Data.TensorDataset

        datasets[split] = Data.TensorDataset(
            input_ids[split], attention_mask[split], token_type_ids[split], torch.LongTensor(y[split])
        )

        loader[split] = Data.DataLoader(
            datasets[split], batch_size=batch_size, shuffle=False
        )
    return loader, y

In [ ]:
from transformers import get_linear_schedule_with_warmup
from transformers import BertForSequenceClassification


def init_objects(
    lr, n_epochs, max_length=16, batch_size=32, test_size=0.2, random_state=2023
):
    loader, _ = init_loader(max_length=max_length, batch_size=batch_size)

    model = AttentionPoolBertClassifier.from_pretrained("bert-base-uncased", num_labels=2)


    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, eps=1e-8)

    total_steps = len(loader["train"]) * n_epochs

    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=total_steps
    )
    return model, loader, optimizer, scheduler


In [ ]:

!pip install optuna
import torch
import optuna
from transformers import logging


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 21.9 MB/s eta 0:00:00


In [ ]:
lr = 2e-5
n_epochs = 1
max_length = 16
batch_size = 32
test_size = 0.2
random_state = 2023
device = torch.device("cuda")

model, loader, optimizer, scheduler = init_objects(
    lr, n_epochs, max_length, batch_size, test_size, random_state
)
model.to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/584k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/69.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/621k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of AttentionPoolBertClassifier were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['attention_layer.bias', 'attention_layer.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttentionPoolBertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

In [ ]:
_, _ = train_eval_loop(
    model, loader, optimizer, scheduler, device, n_epochs=n_epochs, seed_val=42
)


======== Epoch 1 / 1 ========
Training...

Average training loss: 0.69
  Training epoch took: 0:00:13

Running Validation...
  Validation ACC: 56.00
  Validation took: 0:00:00


In [ ]:

param_dict = {
    "lr": [1e-6, 3e-5],
    "n_epochs": [2,3,4],
    "max_length": [16, 32, 64],
    "batch_size": [16, 32,64],
}


class BertObjective:
    def __init__(self, d, device):
        self.d = d
        self.device = device

    def __call__(self, trial: optuna.trial.Trial):
        self.lr = trial.suggest_float("lr", self.d["lr"][0], self.d["lr"][1], log=True)
        self.n_epochs = trial.suggest_categorical("n_epochs", self.d["n_epochs"])
        self.max_length = trial.suggest_categorical("max_length", self.d["max_length"])
        self.batch_size = trial.suggest_categorical("batch_size", self.d["batch_size"])

        model, loader, optimizer, scheduler = init_objects(
            self.lr, self.n_epochs, self.max_length, self.batch_size
        )
        model.to(self.device)
        val_acc, _ = train_eval_loop(
            model, loader, optimizer, scheduler, self.device, self.n_epochs
        )

        return val_acc


device = torch.device("cuda")
study = optuna.create_study(study_name="Stduy 0", direction="maximize")
study.optimize(BertObjective(param_dict, device), n_trials=24)


[I 2025-04-07 12:47:08,819] A new study created in memory with name: Stduy 0
Some weights of MyBertMaxPoolClassifier were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



======== Epoch 1 / 2 ========
Training...

Average training loss: 0.72
  Training epoch took: 0:00:12

Running Validation...
  Validation ACC: 51.14
  Validation took: 0:00:00

======== Epoch 2 / 2 ========
Training...

Average training loss: 0.69
  Training epoch took: 0:00:24

Running Validation...


[I 2025-04-07 12:47:35,951] Trial 0 finished with value: 50.95899470899471 and parameters: {'lr': 4.75181067704919e-06, 'n_epochs': 2, 'max_length': 64, 'batch_size': 32}. Best is trial 0 with value: 50.95899470899471.


  Validation ACC: 50.96
  Validation took: 0:00:00


Some weights of MyBertMaxPoolClassifier were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

  Validation ACC: 61.03
  Validation took: 0:00:00


Some weights of MyBertMaxPoolClassifier were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



======== Epoch 1 / 4 ========
Training...

Average training loss: 0.70
  Training epoch took: 0:00:12

Running Validation...
  Validation ACC: 56.68
  Validation took: 0:00:00

======== Epoch 2 / 4 ========
Training...

Average training loss: 0.62
  Training epoch took: 0:00:24

Running Validation...
  Validation ACC: 59.28
  Validation took: 0:00:00

======== Epoch 3 / 4 ========
Training...

Average training loss: 0.54
  Training epoch took: 0:00:36

Running Validation...
  Validation ACC: 60.52
  Validation took: 0:00:00

======== Epoch 4 / 4 ========
Training...

Average training loss: 0.48
  Training epoch took: 0:00:48

Running Validation...


[I 2025-04-07 13:05:49,593] Trial 23 finished with value: 60.16865079365079 and parameters: {'lr': 2.8641112391992403e-05, 'n_epochs': 4, 'max_length': 64, 'batch_size': 32}. Best is trial 21 with value: 61.37566137566137.


  Validation ACC: 60.17
  Validation took: 0:00:00


In [ ]:

# Train again with best parameters
lr = study.best_params["lr"]
n_epochs = study.best_params["n_epochs"]
max_length = study.best_params["max_length"]
batch_size = study.best_params["batch_size"]

model, loader, optimizer, scheduler = init_objects(lr, n_epochs, max_length, batch_size)
model.to(device)
val_acc, _ = train_eval_loop(model, loader, optimizer, scheduler, device, n_epochs)
# Obtain Test Results
# %%


Some weights of MyBertMaxPoolClassifier were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



======== Epoch 1 / 4 ========
Training...

Average training loss: 0.71
  Training epoch took: 0:00:12

Running Validation...
  Validation ACC: 59.46
  Validation took: 0:00:00

======== Epoch 2 / 4 ========
Training...

Average training loss: 0.63
  Training epoch took: 0:00:24

Running Validation...
  Validation ACC: 59.09
  Validation took: 0:00:00

======== Epoch 3 / 4 ========
Training...

Average training loss: 0.56
  Training epoch took: 0:00:36

Running Validation...
  Validation ACC: 60.68
  Validation took: 0:00:00

======== Epoch 4 / 4 ========
Training...

Average training loss: 0.52
  Training epoch took: 0:00:48

Running Validation...
  Validation ACC: 61.38
  Validation took: 0:00:00


In [ ]:
import pandas as pd

best_params = {
    "Learning Rate": study.best_params["lr"],
    "Epochs": study.best_params["n_epochs"],
    "Max Length": study.best_params["max_length"],
    "Batch Size": study.best_params["batch_size"]
}

best_params_df = pd.DataFrame([best_params])
print(best_params_df.to_string(index=False))


 Learning Rate  Epochs  Max Length  Batch Size
      0.000024       4          64          32


In [ ]:

param_dict = {
    "lr": [1e-6, 3e-5],
    "n_epochs": [2,3,4],
    "max_length": [16, 32, 64],
    "batch_size": [16, 32,64],
}


class BertObjective:
    def __init__(self, d, device):
        self.d = d
        self.device = device

    def __call__(self, trial: optuna.trial.Trial):
        self.lr = trial.suggest_float("lr", self.d["lr"][0], self.d["lr"][1], log=True)
        self.n_epochs = trial.suggest_categorical("n_epochs", self.d["n_epochs"])
        self.max_length = trial.suggest_categorical("max_length", self.d["max_length"])
        self.batch_size = trial.suggest_categorical("batch_size", self.d["batch_size"])

        model, loader, optimizer, scheduler = init_objects(
            self.lr, self.n_epochs, self.max_length, self.batch_size
        )
        model.to(self.device)
        val_acc, _ = train_eval_loop(
            model, loader, optimizer, scheduler, self.device, self.n_epochs
        )

        return val_acc


device = torch.device("cuda")
study = optuna.create_study(study_name="Stduy 0", direction="maximize")
study.optimize(BertObjective(param_dict, device), n_trials=24)


[I 2025-04-07 16:44:21,399] A new study created in memory with name: Stduy 0
Some weights of AttentionPoolBertClassifier were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['attention_layer.bias', 'attention_layer.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



======== Epoch 1 / 3 ========
Training...

Average training loss: 0.69
  Training epoch took: 0:00:13

Running Validation...
  Validation ACC: 55.56
  Validation took: 0:00:00

======== Epoch 2 / 3 ========
Training...

Average training loss: 0.68
  Training epoch took: 0:00:26

Running Validation...
  Validation ACC: 57.64
  Validation took: 0:00:00

======== Epoch 3 / 3 ========
Training...

Average training loss: 0.66
  Training epoch took: 0:00:40

Running Validation...


[I 2025-04-07 16:45:03,966] Trial 0 finished with value: 56.87500000000001 and parameters: {'lr': 4.360915124784374e-06, 'n_epochs': 3, 'max_length': 64, 'batch_size': 16}. Best is trial 0 with value: 56.87500000000001.


  Validation ACC: 56.88
  Validation took: 0:00:00


Some weights of AttentionPoolBertClassifier were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['attention_layer.bias', 'attention_layer.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

  Validation ACC: 59.26
  Validation took: 0:00:01


Some weights of AttentionPoolBertClassifier were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['attention_layer.bias', 'attention_layer.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



======== Epoch 1 / 4 ========
Training...

Average training loss: 0.70
  Training epoch took: 0:00:11

Running Validation...
  Validation ACC: 54.55
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...

Average training loss: 0.68
  Training epoch took: 0:00:22

Running Validation...
  Validation ACC: 56.12
  Validation took: 0:00:01

======== Epoch 3 / 4 ========
Training...

Average training loss: 0.68
  Training epoch took: 0:00:33

Running Validation...
  Validation ACC: 57.34
  Validation took: 0:00:01

======== Epoch 4 / 4 ========
Training...

Average training loss: 0.67
  Training epoch took: 0:00:44

Running Validation...


[I 2025-04-07 17:01:33,764] Trial 23 finished with value: 56.08630952380953 and parameters: {'lr': 5.329121742275755e-06, 'n_epochs': 4, 'max_length': 64, 'batch_size': 64}. Best is trial 10 with value: 61.78571428571429.


  Validation ACC: 56.09
  Validation took: 0:00:01


In [ ]:

# Train again with best parameters
lr = study.best_params["lr"]
n_epochs = study.best_params["n_epochs"]
max_length = study.best_params["max_length"]
batch_size = study.best_params["batch_size"]

model, loader, optimizer, scheduler = init_objects(lr, n_epochs, max_length, batch_size)
model.to(device)
val_acc, _ = train_eval_loop(model, loader, optimizer, scheduler, device, n_epochs)
# Obtain Test Results
# %%


Some weights of AttentionPoolBertClassifier were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['attention_layer.bias', 'attention_layer.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



======== Epoch 1 / 4 ========
Training...

Average training loss: 0.70
  Training epoch took: 0:00:11

Running Validation...
  Validation ACC: 54.90
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...

Average training loss: 0.69
  Training epoch took: 0:00:22

Running Validation...
  Validation ACC: 56.15
  Validation took: 0:00:01

======== Epoch 3 / 4 ========
Training...

Average training loss: 0.69
  Training epoch took: 0:00:33

Running Validation...
  Validation ACC: 56.46
  Validation took: 0:00:01

======== Epoch 4 / 4 ========
Training...

Average training loss: 0.69
  Training epoch took: 0:00:44

Running Validation...
  Validation ACC: 56.46
  Validation took: 0:00:01


In [ ]:
import pandas as pd

best_params = {
    "Learning Rate": study.best_params["lr"],
    "Epochs": study.best_params["n_epochs"],
    "Max Length": study.best_params["max_length"],
    "Batch Size": study.best_params["batch_size"]
}

best_params_df = pd.DataFrame([best_params])
print(best_params_df.to_string(index=False))


 Learning Rate  Epochs  Max Length  Batch Size
      0.000001       4          16          64
